In [ ]:
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
import time
import os
import re
import json
from urllib.parse import urljoin, urlparse
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from collections import deque

class DeepNestedCrawler:
    """深度爬取，會進入每個景點/餐廳/景區的詳細頁面"""
    
    def __init__(self, output_dir="japan_deep_corpus", progress_file="deep_crawl_progress.json", fresh_start=False):
        self.base_url = "https://www.japan-guide.com"
        self.entry_url = "https://www.japan-guide.com/e/e690.html"
        self.output_dir = output_dir
        self.progress_file = progress_file
        
        if fresh_start:
            self.visited_urls = set()
            print("🔥 重新開始深度爬取")
        else:
            self.visited_urls = self.load_progress()
        
        self.to_visit = deque()
        
        # 統計資訊
        self.stats = {
            'pages_with_content': 0,
            'pages_visited': 0,
            'links_discovered': 0
        }
        
        # Setup session
        self.session = requests.Session()
        retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
        self.session.mount('https://', HTTPAdapter(max_retries=retries))
        
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"
        }
        
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def load_progress(self):
        if os.path.exists(self.progress_file):
            try:
                with open(self.progress_file, 'r') as f:
                    data = json.load(f)
                    return set(data.get('visited_urls', []))
            except:
                return set()
        return set()
    
    def save_progress(self):
        """儲存進度和統計"""
        data = {
            'visited_urls': list(self.visited_urls),
            'to_visit': list(self.to_visit),
            'stats': self.stats
        }
        with open(self.progress_file, 'w') as f:
            json.dump(data, f, indent=2)

    def normalize_url(self, url):
        parsed = urlparse(url)
        return f"{parsed.scheme}://{parsed.netloc}{parsed.path}"

    def is_valid_content_url(self, url):
        """判斷是否為有效的內容頁面"""
        # 只爬取 /e/eXXXX.html 格式的頁面
        if not '/e/e' in url:
            return False
        
        # 排除某些不需要的頁面
        exclude_patterns = [
            '/forum/',
            '/login',
            '/survey',
            'javascript:',
            '#',
            '.jpg',
            '.png',
            '.gif',
            '.pdf'
        ]
        
        for pattern in exclude_patterns:
            if pattern in url:
                return False
        
        return True

    def extract_all_links(self, soup, current_url):
        """提取所有有效連結"""
        links = set()
        
        for a in soup.find_all('a', href=True):
            href = a['href']
            
            # 處理相對路徑
            if href.startswith('/e/e'):
                full_url = urljoin(self.base_url, href)
            elif href.startswith('e/e'):
                full_url = urljoin(current_url, href)
            elif 'japan-guide.com/e/e' in href:
                full_url = href
            else:
                continue
            
            # 標準化 URL
            normalized = self.normalize_url(full_url)
            
            # 驗證是否為有效的內容 URL
            if self.is_valid_content_url(normalized):
                links.add(normalized)
        
        return links

    def clean_content(self, soup):
        """清理不需要的元素"""
        noise_selectors = [
            'script', 'style', 'noscript', 'svg',
            'nav', 'footer', 'header', 'form',
            '#header', '#footer', '#navigation', '#sidebar',
            '[role="navigation"]', '[role="banner"]', '[role="complementary"]',
            '.navigation_bar', '.social_links', '.breadcrumb',
            '[id*="ad"]', '[class*="ad_"]', '[class*="advertisement"]',
            '.booking-widget', '.hotel-widget', '.survey',
            'iframe', '[class*="widget"]', '[class*="banner"]',
            'img[src*="logo"]', 'a[href*="login"]', 
            '.feedback', '[class*="newsletter"]',
        ]
        
        for selector in noise_selectors:
            for element in soup.select(selector):
                element.decompose()
        
        return soup

    def extract_main_content(self, soup):
        """提取主要內容"""
        body = soup.find('body')
        if not body:
            return None
        
        body_html = str(body)
        content_soup = BeautifulSoup(body_html, 'html.parser')
        return self.clean_content(content_soup)

    def extract_metadata(self, soup, url):
        """提取元數據"""
        metadata = {}
        
        # 標題
        if soup.title:
            title_text = soup.title.string
            if ' - ' in title_text:
                metadata['title'] = title_text.split(' - ')[0].strip()
            else:
                metadata['title'] = title_text.strip()
        
        # 描述
        desc = soup.find('meta', attrs={'name': 'description'})
        if desc and desc.get('content'):
            metadata['description'] = desc['content']
        
        # 關鍵字
        keywords = soup.find('meta', attrs={'name': 'keywords'})
        if keywords and keywords.get('content'):
            metadata['keywords'] = keywords['content']
        
        # 判斷內容類型
        path = urlparse(url).path
        content_type = 'general'
        
        # 從 URL 判斷類型
        if '/e/e2' in path:
            content_type = 'city_guide'
        elif '/e/e3' in path:
            content_type = 'attraction'
        elif '/e/e4' in path:
            content_type = 'regional_attraction'
        elif '/e/e5' in path or '/e/e6' in path:
            content_type = 'regional_guide'
        elif '/e/e7' in path:
            content_type = 'onsen_or_rural'
        elif '/e/e8' in path:
            content_type = 'other_region'
        
        metadata['content_type'] = content_type
        
        return metadata

    def clean_markdown(self, content_md):
        """清理 markdown 內容"""
        # 移除圖片佔位符
        content_md = re.sub(r'!\[\]\(\)', '', content_md)
        content_md = re.sub(r'!\[\]\([^\)]*\)', '', content_md)
        
        # 移除星號評分
        content_md = re.sub(r'★+', '', content_md)
        
        # 移除空的連結
        content_md = re.sub(r'\[!\[\]\([^\)]*\)\]\([^\)]*\)', '', content_md)
        
        # 清理空行
        content_md = re.sub(r'\n{3,}', '\n\n', content_md)
        
        # 移除頁尾
        content_md = re.sub(r'\*\*Questions\?\*\*.*?forum.*?\n', '', content_md, flags=re.IGNORECASE)
        content_md = re.sub(r'Page last updated:.*', '', content_md)
        content_md = re.sub(r'### English\s*\*.*', '', content_md, flags=re.DOTALL)
        content_md = re.sub(r'### Japanese\s*\*.*', '', content_md, flags=re.DOTALL)
        
        # 移除 booking.com 和其他廣告區塊
        content_md = re.sub(r'Top rated in.*?(?=\n##|\n\*\*|\Z)', '', content_md, flags=re.DOTALL)
        content_md = re.sub(r'powered by.*?Booking\.com.*', '', content_md, flags=re.IGNORECASE)
        content_md = re.sub(r'\[View on Booking\.com\].*', '', content_md)
        content_md = re.sub(r'Recommended in.*?JapanEatinerary.*', '', content_md, flags=re.DOTALL)
        
        return content_md.strip()

    def scrape_page(self, url):
        """爬取單個頁面"""
        try:
            res = self.session.get(url, headers=self.headers, timeout=15)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, 'html.parser')
            
            # 提取所有連結
            new_links = self.extract_all_links(soup, url)
            self.stats['links_discovered'] += len(new_links)
            
            # 提取內容
            content_area = self.extract_main_content(soup)
            
            if not content_area:
                return new_links, None
            
            # 檢查內容長度
            text_content = content_area.get_text(strip=True)
            if len(text_content) < 100:
                return new_links, None
            
            # 提取元數據
            metadata = self.extract_metadata(soup, url)
            
            # 轉換為 markdown
            content_md = md(str(content_area), heading_style="ATX")
            content_md = self.clean_markdown(content_md)
            
            # 建立 frontmatter
            frontmatter = "---\n"
            frontmatter += f"url: {url}\n"
            
            for key, value in metadata.items():
                safe_value = str(value).replace(':', '\\:').replace('\n', ' ')
                frontmatter += f"{key}: {safe_value}\n"
            
            # 統計資訊
            word_count = len(text_content.split())
            char_count = len(text_content)
            frontmatter += f"word_count: {word_count}\n"
            frontmatter += f"char_count: {char_count}\n"
            frontmatter += f"crawled_at: {time.strftime('%Y-%m-%d %H:%M:%S')}\n"
            frontmatter += "---\n\n"
            
            return new_links, frontmatter + content_md
            
        except Exception as e:
            print(f"❌ 錯誤 {url}: {e}")
            return set(), None

    def save_content(self, url, content):
        """儲存內容"""
        path_part = urlparse(url).path
        filename = re.sub(r'[\\/*?:"<>|]', "_", path_part.strip("/")) + ".md"
        filepath = os.path.join(self.output_dir, filename)
        
        try:
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(content)
            return True
        except Exception as e:
            print(f"❌ 儲存失敗 {filename}: {e}")
            return False

    def crawl(self, max_pages=None, depth='all'):
        """
        主爬取循環
        
        Args:
            max_pages: 最大爬取頁數，None 表示無限制
            depth: 'all' = 爬取所有發現的連結
                   'one_level' = 只爬取第一層（主頁面）
                   'two_levels' = 爬取兩層（主頁面 + 子頁面）
        """
        # 開始 URL
        self.to_visit.append(self.normalize_url(self.entry_url))
        
        print(f"🚀 開始深度爬取")
        print(f"📂 輸出目錄: {self.output_dir}")
        print(f"🔍 爬取深度: {depth}")
        print(f"🔄 已訪問: {len(self.visited_urls)} 頁\n")
        
        level = 0  # 追蹤當前層級
        pages_in_current_level = len(self.to_visit)
        pages_processed_in_level = 0
        
        while self.to_visit and (max_pages is None or self.stats['pages_with_content'] < max_pages):
            url = self.to_visit.popleft()
            
            # 跳過已訪問
            if url in self.visited_urls:
                continue
            
            self.stats['pages_visited'] += 1
            
            # 顯示進度
            level_info = f"[Level {level}]" if depth != 'all' else ""
            print(f"{level_info} [{self.stats['pages_visited']}] 爬取: {url}")
            
            # 爬取頁面
            new_links, content = self.scrape_page(url)
            
            # 儲存內容
            if content:
                if self.save_content(url, content):
                    print(f"✅ 成功 (發現 {len(new_links)} 個新連結)")
                    self.stats['pages_with_content'] += 1
            
            # 標記為已訪問
            self.visited_urls.add(url)
            
            # 根據深度設定決定是否加入新連結
            if depth == 'all':
                # 加入所有新連結
                for link in new_links:
                    if link not in self.visited_urls and link not in self.to_visit:
                        self.to_visit.append(link)
            elif depth == 'one_level':
                # 只爬取初始頁面，不加入新連結
                pass
            elif depth == 'two_levels':
                # 只加入前兩層的連結
                if level < 2:
                    for link in new_links:
                        if link not in self.visited_urls and link not in self.to_visit:
                            self.to_visit.append(link)
            
            # 追蹤層級
            pages_processed_in_level += 1
            if pages_processed_in_level >= pages_in_current_level:
                level += 1
                pages_in_current_level = len(self.to_visit)
                pages_processed_in_level = 0
                if depth != 'all' and pages_in_current_level > 0:
                    print(f"\n🔽 進入 Level {level} (待爬取: {pages_in_current_level})\n")
            
            # 定期儲存進度
            if self.stats['pages_visited'] % 50 == 0:
                self.save_progress()
                print(f"\n💾 進度已儲存")
                print(f"   已爬取: {self.stats['pages_with_content']}")
                print(f"   待處理: {len(self.to_visit)}")
                print(f"   發現連結: {self.stats['links_discovered']}\n")
            
            # 禮貌延遲
            time.sleep(1.2)
        
        # 最終儲存
        self.save_progress()
        
        print("\n" + "="*60)
        print("🎉 爬取完成!")
        print("="*60)
        print(f"📊 成功爬取: {self.stats['pages_with_content']} 頁")
        print(f"📝 總共訪問: {self.stats['pages_visited']} 頁")
        print(f"🔗 發現連結: {self.stats['links_discovered']} 個")
        print(f"📋 待處理: {len(self.to_visit)} 頁")


if __name__ == "__main__":
    
    # 完整深度爬取（推薦，會爬取所有景點詳細頁面）
    crawler = DeepNestedCrawler(fresh_start=True)
    crawler.crawl(depth='all')

🔥 重新開始深度爬取
🚀 開始深度爬取
📂 輸出目錄: japan_deep_corpus
🔍 爬取深度: all
🔄 已訪問: 0 頁

 [1] 爬取: https://www.japan-guide.com/e/e690.html
✅ 成功 (發現 36 個新連結)
 [2] 爬取: https://www.japan-guide.com/e/e2160.html
✅ 成功 (發現 48 個新連結)
 [3] 爬取: https://www.japan-guide.com/e/e2166.html
✅ 成功 (發現 54 個新連結)
 [4] 爬取: https://www.japan-guide.com/e/e623.html
✅ 成功 (發現 51 個新連結)
 [5] 爬取: https://www.japan-guide.com/e/e440.html
✅ 成功 (發現 34 個新連結)
 [6] 爬取: https://www.japan-guide.com/e/e2058.html
✅ 成功 (發現 49 個新連結)
 [7] 爬取: https://www.japan-guide.com/e/e5200.html
✅ 成功 (發現 67 個新連結)
 [8] 爬取: https://www.japan-guide.com/e/e627.html
✅ 成功 (發現 102 個新連結)
 [9] 爬取: https://www.japan-guide.com/e/e460.html
✅ 成功 (發現 34 個新連結)
 [10] 爬取: https://www.japan-guide.com/e/e2427.html
✅ 成功 (發現 35 個新連結)
 [11] 爬取: https://www.japan-guide.com/e/e623a.html
✅ 成功 (發現 228 個新連結)
 [12] 爬取: https://www.japan-guide.com/e/e2158.html
✅ 成功 (發現 125 個新連結)
 [13] 爬取: https://www.japan-guide.com/e/e2165.html
✅ 成功 (發現 60 個新連結)
 [14] 爬取: https://www.japan-guide.com/e/e217